In [1]:
os.system("python setup.py build_ext --inplace")

1

In [2]:
from tensorflow import keras
import tensorflow as tf
from keras_retinanet import models
import keras_retinanet
from keras_retinanet.preprocessing.trainClasse import OsuGenerator
import numpy as np
from PIL import Image

import os

sys.path.append('..')
from utilitaire.imgAiTrainer.imgAiTrainer import Trainer
tr = Trainer()


In [3]:
def generateDataSet(nb,x,y) :
    lImg = []
    lLabel = []
    lBox = []
    for j in range(nb):
        cercleT = tr.createMultiCercle((x,y),3)
        img = tf.keras.preprocessing.image.load_img("../../Assets/imgAiTrainer/cercle.png",target_size=(120,160))
        img = keras.preprocessing.image.img_to_array(img)
        lImg.append(img)
        lLabelTmp = []
        lBoxTmp = []
        box2 = np.empty((len(cercleT[0]),4))
        for i in range(len(cercleT[0])):
            box = np.zeros((4,))
            box[0] = (cercleT[0][i-1]-cercleT[3][i-1])/5
            box[1] = (cercleT[1][i-1]-cercleT[3][i-1])/5
            box[2] = (cercleT[0][i-1]+cercleT[3][i-1])/5
            box[3] = (cercleT[1][i-1]+cercleT[3][i-1])/5
            box2[i, :] = box
            #lBoxTmp.append((cercleT[0][i-1]-cercleT[3][i-1],cercleT[1][i-1]-cercleT[3][i-1],cercleT[0][i-1]+cercleT[3][i-1],cercleT[1][i-1]+cercleT[3][i-1]))
            lLabelTmp.append(0)
        lLabel.append(np.array(lLabelTmp))
        lBox.append(box2)
    lImg = np.array(lImg)
    lLabel = np.array(lLabel)
    return lImg, lBox, lLabel

In [4]:
model = models.backbone('resnet50').retinanet(num_classes=3)

In [5]:
model.compile(
    loss={
        'regression'    : keras_retinanet.losses.smooth_l1(),
        'classification': keras_retinanet.losses.focal()
    },
    optimizer=keras.optimizers.Adam(lr=1e-5, clipnorm=0.001)
)
#model.summary()

In [6]:
img, box, labels = generateDataSet(10,600,800)

In [8]:
print(box[0])
print(labels[0])
box[0].shape

[]
[]


(0, 4)

In [9]:
trainGenerator = OsuGenerator(img, box, labels)
model.fit_generator(
        generator = trainGenerator,
        steps_per_epoch=50,
        epochs=20,
        verbose=1
    )

Epoch 1/20


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node retinanet/conv1/Conv2D (defined at <ipython-input-9-72753d1de7fa>:2) ]] [Op:__inference_train_function_15371]

Function call stack:
train_function
